# Customer Segmentation using RFM analysis
### Method 2: Defining Functions

In [1]:
## Import appropriate modules
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
plt.style.use('seaborn-white')
import seaborn as sns 
import datetime as dt

<Figure size 720x576 with 0 Axes>

In [2]:
# load the data set
df = pd.read_excel('data_for_RFM_score_analysis.xlsx')
print(df.head()) 

                                     id       date   sales
0  000173c5-978c-4b52-b7a4-5ebf974deb86 2020-08-13  1690.0
1  000173c5-978c-4b52-b7a4-5ebf974deb86 2020-08-14  6145.0
2  000173c5-978c-4b52-b7a4-5ebf974deb86 2020-08-15  4550.0
3  000173c5-978c-4b52-b7a4-5ebf974deb86 2020-08-17  1270.0
4  000173c5-978c-4b52-b7a4-5ebf974deb86 2020-08-20  3830.0


In [3]:
# convert the date column to datetime data type
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804659 entries, 0 to 804658
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   id      804659 non-null  object        
 1   date    804659 non-null  datetime64[ns]
 2   sales   804659 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 18.4+ MB


In [4]:
# create a snapshot date with today's date
print('Maximum Date:', df['date'].max()) # Max Date: 2020-09-25 00:00:00

snapshot_date = max(df.date) + dt.timedelta(days=1)
print('Current Date:',snapshot_date) # 2020-09-26 00:00:00

Maximum Date: 2020-09-25 00:00:00
Current Date: 2020-09-26 00:00:00


In [5]:
# create functions to get recency and tenure
def get_recency(x):
    last_purchase = x.max()
    return (snapshot_date - last_purchase).days

def get_tenure(x):
    first_purchase = x.min()
    return (snapshot_date - first_purchase).days

# aggregate data by the customers
customers = df.groupby('id').agg(
    recency=('date', get_recency),
    tenure=('date', get_tenure),
    frequency=('id', 'count'),
    total_value=('sales', 'sum'),
    mean_value=('sales', 'mean')
    )
# show 5 samples of the grouped dataframe
print(customers.sample(5))

                                      recency  tenure  frequency  total_value  \
id                                                                              
60d4746e-c945-4ad7-ad3d-7240487edb8b       31      31          1       1950.0   
c73b736c-6042-4ba8-947a-8183ba1f3076       29     161         16      14925.0   
948b966c-7a24-4357-9591-c1656a6c9a34        1      46         11       3570.0   
d0e08372-1044-48ad-9636-faae5a8f12f1       92     159          9      16115.0   
5b849fe4-7f36-4242-8d3a-466afa9de433       44     161          7       6320.0   

                                       mean_value  
id                                                 
60d4746e-c945-4ad7-ad3d-7240487edb8b  1950.000000  
c73b736c-6042-4ba8-947a-8183ba1f3076   932.812500  
948b966c-7a24-4357-9591-c1656a6c9a34   324.545455  
d0e08372-1044-48ad-9636-faae5a8f12f1  1790.555556  
5b849fe4-7f36-4242-8d3a-466afa9de433   902.857143  


### Remark 1
We have added another two columns:
* the tenure, which represents the time since the customer first purchase 
* mean value

### RFM Segments
* The next thing we need to do is to segment the __recency__, __frequency__, and __total_value__ into the categories.
* For our use case, we decided to split each feature into 4 quartiles that roughly divide the sample into 4 segments of equal proportion. 
* We have called these scores __R__, __F__, and __M__ respectively.

In [8]:
# use only the necessary columns
customers = customers.reset_index() # re-set index for the next caclculation
#rfm = customers[['id', 'recency', 'frequency', 'total_value']]
print(customers.head())

   index                                    id  recency  tenure  frequency  \
0      0  000173c5-978c-4b52-b7a4-5ebf974deb86        1     161         54   
1      1  0001b0ce-f323-49b5-b381-3348c7a001ab        1      85         54   
2      2  00020337-5321-4d6e-83af-67905edd8006        8      49          4   
3      3  00066d09-6e2e-4104-b03b-6927490e1972       16     163         84   
4      4  00069d0b-c994-4555-a30e-0c0799d77fe1       10     175         48   

   total_value   mean_value  
0     149415.0  2766.944444  
1     266800.0  4940.740741  
2      13385.0  3346.250000  
3      75388.0   897.476190  
4      75630.0  1575.625000  


In [27]:
# recency quartile segmentation
r_labels = range(4, 0, -1)
recency = customers['recency']
r_quartiles, bins = pd.qcut(recency, 4, labels=r_labels, retbins=True)
customers = customers.assign(R=r_quartiles.values)

# frequency quartile segmentation
f_labels = range(1, 5)
frequency = customers['frequency'].rank(method='first') # rank to deal with duplicate values
f_quartiles, bins = pd.qcut(frequency, 4, labels=f_labels, retbins=True)
customers = customers.assign(F = f_quartiles.values)

# monetary value quartile segmentation
m_labels = range(1, 5)
monetary = customers['total_value']
m_quartiles, bins = pd.qcut(monetary, 4, labels=m_labels, retbins=True)
customers = customers.assign(M = m_quartiles.values)

customers['RFMClass'] = customers["R"].astype(str) + customers["F"].astype(str) + customers["M"].astype(str)
customers['score'] = customers['R'].astype(int) + customers['F'].astype(int) + customers['M'].astype(int)

print(customers.head())

   index                                    id  recency  tenure  frequency  \
0      0  000173c5-978c-4b52-b7a4-5ebf974deb86        1     161         54   
1      1  0001b0ce-f323-49b5-b381-3348c7a001ab        1      85         54   
2      2  00020337-5321-4d6e-83af-67905edd8006        8      49          4   
3      3  00066d09-6e2e-4104-b03b-6927490e1972       16     163         84   
4      4  00069d0b-c994-4555-a30e-0c0799d77fe1       10     175         48   

   total_value   mean_value  R  F  M RFMClass  Score  score  
0     149415.0  2766.944444  4  4  4      444     12     12  
1     266800.0  4940.740741  4  4  4      444     12     12  
2      13385.0  3346.250000  3  2  2      322      7      7  
3      75388.0   897.476190  2  4  4      244     10     10  
4      75630.0  1575.625000  3  4  4      344     11     11  


### Remark 2
* To simplify the analysis, it is important that we combine the 3 different scores (R, F, and M) to create a single metric.
* There are a few approaches available:

1. The first one is to create an RFM Segment by concatenating the 3 digits from the individual scores to form a 3 character string that goes from 111 (lowest possible score in all three metrics) to 444 (highest possible score in all three metrics). The drawback of this method is the creation of many distinct segments (4x4x4 = 64 segments) which are not so easy to differentiate and prioritize (who is more valuable, a 432 or a 234 customer?).

2. Another possibility is to sum the 3 individual scores to create the RFM Score, a number ranging from 3 (lowest possible score in all metrics) to 12 (highest possible score in all metrics). Here the drawback is that customers with different buying habits (e.g. from different RFM Segments) can fall on the same score bins. For example, both customers in segments 431 and 134 would get a score of 8. On the other hand, we end up with less distinct scores to compare (4+4+4 = 12 scores), each of them with equal relevance.

### Remark 3
We can further divide the customers into RFM tiers by binning together ranges of scores. For example, we can say that customers with scores ranging from 3 to 5 are Bronze, from 5 to 9 are Silver and from 9 to 12 are Gold.

In [ ]:
RFM_Segment['TotalScore'] = RFM_Segment['R_Quartile'] + RFM_Segment['F_Quartile'] + RFM_Segment['M_Quartile']

In [31]:
# group into different tiers
def get_tier(df):
    if df['score'] >= 9:
        return 'Gold'
    elif (df['score'] >= 5) and (df['score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'
customers['tier'] = customers.apply(get_tier, axis=1)
# save to excel
customers.to_excel('Report_RFM_Segments_Method_2.xlsx')
print(customers.head())

   index                                    id  recency  tenure  frequency  \
0      0  000173c5-978c-4b52-b7a4-5ebf974deb86        1     161         54   
1      1  0001b0ce-f323-49b5-b381-3348c7a001ab        1      85         54   
2      2  00020337-5321-4d6e-83af-67905edd8006        8      49          4   
3      3  00066d09-6e2e-4104-b03b-6927490e1972       16     163         84   
4      4  00069d0b-c994-4555-a30e-0c0799d77fe1       10     175         48   

   total_value   mean_value  R  F  M RFMClass  Score  score    tier  
0     149415.0  2766.944444  4  4  4      444     12     12    Gold  
1     266800.0  4940.740741  4  4  4      444     12     12    Gold  
2      13385.0  3346.250000  3  2  2      322      7      7  Silver  
3      75388.0   897.476190  2  4  4      244     10     10    Gold  
4      75630.0  1575.625000  3  4  4      344     11     11    Gold  


In [37]:
#### Distribution of customers under different tiers
customers['tier'].value_counts()

Gold      14504
Silver    12781
Bronze     7612
Name: tier, dtype: int64

#### References
1. https://towardsdatascience.com/divide-and-conquer-segment-your-customers-using-rfm-analysis-68aee749adf6